In [4]:
import pandas as pd
from utils import *
import re
import os

In [5]:
headers = {
    'authority': 'www.transfermarkt.com.br',
    'accept': '*/*',
    'accept-language': 'pt-BR,pt;q=0.9',
    # 'cookie': '_tmlpu=5; _sp_v1_ss=1:H4sIAAAAAAAAAItWqo5RKimOUbKKhjHySnNydGKUUpHYJWCJ6traWFwSSjqYBmFn5IEYBrhNoqKEUiwAqTzxSd8AAAA%3D; _sp_v1_p=875; _sp_v1_data=746454; _sp_su=false; euconsent-v2=CP3wTQAP3wTQAAGABCENAeEsAP_gAEPAAAKIJDgBdDJECCFAIXBaAOsQKIEVUVABAEAAAAABACABQAAAIAQCkAAIAACAAigAARAAIEQAAAAAAAAABAAAAIAAIAEEAAAQgAAIIAAAAAAAAABAAAAIAAAAQAAAgAABAAQAkACIAAIAUEAAAAACAAAQAIgAAIAAAgAAAAAAAAAAAAIIICgAAAAAAAAAAAACABAAAAAIH7wEQAFAAOAEUAI4AcgBCACIgE7ALEAXUA14B2wF0AMEAZCAyYB-4BoSAqABUADgAIAAZAA0ACIAEwAJ4AZgA3gB6AD8AIQAQwAmgBlAD9AKeAo8BeYDJAoAIARQC6A0AEAp46A4ABUADgAIAAZAA0ACIAEwAJ8AXABdADEAGYAN4AegA_ACGAE0AMoAfoBFgCngFiAReAo8BVgC8wGSAMsAcWPAAgCKEQAQCnkIBAATAAuABiADeAHoARwBTwFWEAAIA5CUAsADgARAAmABcADEAIYAp4CLwFHgLzAZISAAgMsLQAgBHAKsKQFAAKgAcABAADQAIgATAAngBiADMAH4AQwAygB-gEWAPaAi8BVgC8wGSAMsKAAwAZADkAJO.YAAAAAAAAAAA; consentUUID=2b918714-7bff-40b6-a4c8-bf4579a973a0_27; TMSESSID=79a54446e857d26e9f36f18f3b796cca',
    'if-modified-since': 'Tue, 02 Jan 2024 23:03:39 GMT',
    'referer': 'https://www.transfermarkt.com.br/john-kennedy/profil/spieler/856085',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Brave";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
}

In [6]:
def extract_player_ids(dataframe):
    fbref_pattern = r"/players/([a-fA-F0-9]+)/"
    tmarkt_pattern = r"/spieler/(\d+)"

    fbref_ids = []
    tmarkt_ids = []

    for _, row in dataframe.iterrows():
        fbref_match = re.search(fbref_pattern, row['UrlFBref'])
        tmarkt_match = re.search(tmarkt_pattern, row['UrlTmarkt'])

        fbref_id = fbref_match.group(1) if fbref_match else None
        tmarkt_id = tmarkt_match.group(1) if tmarkt_match else None

        fbref_ids.append(fbref_id)
        tmarkt_ids.append(tmarkt_id)

    dataframe['FBref_Player_ID'] = fbref_ids
    dataframe['Tmarkt_Player_ID'] = tmarkt_ids

    return dataframe

def create_player_images_url(dataframe):
    dataframe["ImageUrl"] = dataframe.apply(lambda row: f"https://www.transfermarkt.com/ceapi/player/{row['Tmarkt_Player_ID']}/images", axis=1)
    return dataframe


In [10]:
fbref_tm_mapping = pd.read_csv(os.path.join("fbref_tm_mapping", "fbref_to_tm_mapping.csv"), encoding="ISO-8859-1")

fbref_tm_mapping = extract_player_ids(fbref_tm_mapping)
fbref_tm_mapping.head(5)

,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBref_Player_ID,Tmarkt_Player_ID
0,A.J. DeLaGarza,https://fbref.com/en/players/171b3c37/AJ-DeLaG...,https://www.transfermarkt.com/a-j-delagarza/pr...,Right-Back,171b3c37,105997
1,AJ Marcucci,https://fbref.com/en/players/20c86a38/AJ-Marcucci,https://www.transfermarkt.com/aj-marcucci/prof...,Goalkeeper,20c86a38,884244
2,Aapo Halme,https://fbref.com/en/players/02b952ce/Aapo-Halme,https://www.transfermarkt.com/aapo-halme/profi...,Centre-Back,02b952ce,322984
3,Aaron Bastiaans,https://fbref.com/en/players/fb979733/Aaron-Ba...,https://www.transfermarkt.com/aaron-bastiaans/...,Left Winger,fb979733,578459
4,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,Centre-Forward,27c01749,434207


In [12]:
fbref_tm_mapping.columns

Index(['PlayerFBref', 'UrlFBref', 'UrlTmarkt', 'TmPos', 'FBref_Player_ID',
       'Tmarkt_Player_ID'],
      dtype='object')

In [23]:
fbref_tm_mapping = create_player_images_url(fbref_tm_mapping)
fbref_tm_mapping.to_csv(os.path.join("fbref_tm_mapping", "fbref_to_tm_mapping_with_images.csv"), encoding="ISO-8859-1")

In [3]:
BASE_DIR = os.path.join('..', 'data')
RAW_DIR = os.path.join(BASE_DIR, 'raw')
ENGINEERED_DIR = os.path.join(BASE_DIR, 'engineered')
SEASON_DIR = os.path.join(RAW_DIR, "2023")
BRAZIL_DIR = os.path.join(SEASON_DIR, "Brazilian Serie A")

In [11]:
brazil_df = pd.read_csv(os.path.join(BRAZIL_DIR, "standard.csv"))
brazil_df.head(5)

,Rk,Player,Nation,Pos,Squad,Age,Born,Playing Time_MP,Playing Time_Starts,Playing Time_Min,...,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Player Link_,Player ID_,Competition
0,1,Nicolás Acevedo,URU,MF,Bahia,23.0,1999.0,31,21,1970,...,0.00,0.00,0.04,0.05,0.10,0.04,0.10,https://fbref.com/en/players/6140be7b/Nicolas-...,6140be7b,Brazilian Serie A
1,2,Luiz Adriano,BRA,FW,Internacional,35.0,1987.0,29,17,1330,...,0.20,0.27,0.18,0.09,0.27,0.18,0.27,https://fbref.com/en/players/2557c196/Luiz-Adr...,2557c196,Brazilian Serie A
2,3,Adriel,BRA,GK,Grêmio,22.0,2001.0,2,2,180,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,https://fbref.com/en/players/11ad100a/Adriel,11ad100a,Brazilian Serie A
3,4,Adryelson,BRA,DF,Botafogo (RJ),24.0,1998.0,35,35,2996,...,0.03,0.06,0.03,0.01,0.04,0.03,0.04,https://fbref.com/en/players/e980e78d/Adryelson,e980e78d,Brazilian Serie A
4,5,Adson,BRA,"FW,MF",Corinthians,22.0,2000.0,15,9,857,...,0.00,0.11,0.04,0.11,0.15,0.04,0.15,https://fbref.com/en/players/eda38706/Adson,eda38706,Brazilian Serie A


In [16]:
brazil_df.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born',
       'Playing Time_MP', 'Playing Time_Starts', 'Playing Time_Min',
       'Playing Time_90s', 'Performance_Gls', 'Performance_Ast',
       'Performance_G+A', 'Performance_G-PK', 'Performance_PK',
       'Performance_PKatt', 'Performance_CrdY', 'Performance_CrdR',
       'Expected_xG', 'Expected_npxG', 'Expected_xAG', 'Expected_npxG+xAG',
       'Progression_PrgC', 'Progression_PrgP', 'Progression_PrgR',
       'Per 90 Minutes_Gls', 'Per 90 Minutes_Ast', 'Per 90 Minutes_G+A',
       'Per 90 Minutes_G-PK', 'Per 90 Minutes_G+A-PK', 'Per 90 Minutes_xG',
       'Per 90 Minutes_xAG', 'Per 90 Minutes_xG+xAG', 'Per 90 Minutes_npxG',
       'Per 90 Minutes_npxG+xAG', 'Player Link_', 'Player ID_', 'Competition'],
      dtype='object')

In [14]:
len(brazil_df)

752

In [18]:
brazil_df_tm_fbref = pd.merge(brazil_df, fbref_tm_mapping_join, how="inner", on="Player ID_")
brazil_df_tm_fbref.head(5)

,Rk,Player,Nation,Pos,Squad,Age,Born,Playing Time_MP,Playing Time_Starts,Playing Time_Min,...,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Player Link_,Player ID_,Competition,UrlTmarkt,TmPos,Tmarkt_Player_ID
0,1,Nicolás Acevedo,URU,MF,Bahia,23.0,1999.0,31,21,1970,...,0.05,0.10,0.04,0.10,https://fbref.com/en/players/6140be7b/Nicolas-...,6140be7b,Brazilian Serie A,https://www.transfermarkt.com/nicolas-acevedo/...,Defensive Midfield,625569
1,2,Luiz Adriano,BRA,FW,Internacional,35.0,1987.0,29,17,1330,...,0.09,0.27,0.18,0.27,https://fbref.com/en/players/2557c196/Luiz-Adr...,2557c196,Brazilian Serie A,https://www.transfermarkt.com/luiz-adriano/pro...,Centre-Forward,48974
2,3,Adriel,BRA,GK,Grêmio,22.0,2001.0,2,2,180,...,0.00,0.00,0.00,0.00,https://fbref.com/en/players/11ad100a/Adriel,11ad100a,Brazilian Serie A,https://www.transfermarkt.com/adriel/profil/sp...,Goalkeeper,808445
3,4,Adryelson,BRA,DF,Botafogo (RJ),24.0,1998.0,35,35,2996,...,0.01,0.04,0.03,0.04,https://fbref.com/en/players/e980e78d/Adryelson,e980e78d,Brazilian Serie A,https://www.transfermarkt.com/adryelson/profil...,Centre-Back,373433
4,5,Adson,BRA,"FW,MF",Corinthians,22.0,2000.0,15,9,857,...,0.11,0.15,0.04,0.15,https://fbref.com/en/players/eda38706/Adson,eda38706,Brazilian Serie A,https://www.transfermarkt.com/adson/profil/spi...,Right Winger,868891


In [19]:
brazil_df_final = create_player_images_url(brazil_df_tm_fbref)
brazil_df_final.head(5)

,Rk,Player,Nation,Pos,Squad,Age,Born,Playing Time_MP,Playing Time_Starts,Playing Time_Min,...,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,Player Link_,Player ID_,Competition,UrlTmarkt,TmPos,Tmarkt_Player_ID,ImageUrl
0,1,Nicolás Acevedo,URU,MF,Bahia,23.0,1999.0,31,21,1970,...,0.10,0.04,0.10,https://fbref.com/en/players/6140be7b/Nicolas-...,6140be7b,Brazilian Serie A,https://www.transfermarkt.com/nicolas-acevedo/...,Defensive Midfield,625569,https://www.transfermarkt.com/ceapi/player/625...
1,2,Luiz Adriano,BRA,FW,Internacional,35.0,1987.0,29,17,1330,...,0.27,0.18,0.27,https://fbref.com/en/players/2557c196/Luiz-Adr...,2557c196,Brazilian Serie A,https://www.transfermarkt.com/luiz-adriano/pro...,Centre-Forward,48974,https://www.transfermarkt.com/ceapi/player/489...
2,3,Adriel,BRA,GK,Grêmio,22.0,2001.0,2,2,180,...,0.00,0.00,0.00,https://fbref.com/en/players/11ad100a/Adriel,11ad100a,Brazilian Serie A,https://www.transfermarkt.com/adriel/profil/sp...,Goalkeeper,808445,https://www.transfermarkt.com/ceapi/player/808...
3,4,Adryelson,BRA,DF,Botafogo (RJ),24.0,1998.0,35,35,2996,...,0.04,0.03,0.04,https://fbref.com/en/players/e980e78d/Adryelson,e980e78d,Brazilian Serie A,https://www.transfermarkt.com/adryelson/profil...,Centre-Back,373433,https://www.transfermarkt.com/ceapi/player/373...
4,5,Adson,BRA,"FW,MF",Corinthians,22.0,2000.0,15,9,857,...,0.15,0.04,0.15,https://fbref.com/en/players/eda38706/Adson,eda38706,Brazilian Serie A,https://www.transfermarkt.com/adson/profil/spi...,Right Winger,868891,https://www.transfermarkt.com/ceapi/player/868...


In [20]:
len(brazil_df_final)

569

In [21]:
import ScraperFC as sfc
import traceback
scraper = sfc.Transfermarkt()
try:
    data = scraper.scrape_all_stats(year=2023, league="Brasileirao")
except:
    # Catch and print any exceptions.
    traceback.print_exc()
finally:
    scraper.close()

AttributeError: 'NoneType' object has no attribute 'name'